In [2]:
import numpy as np
from scipy.sparse import diags, csr_matrix
from scipy.sparse.linalg import norm
from scipy.optimize import minimize
from tensorflow.keras.datasets import mnist

# Load and preprocess MNIST data (using only the first 1000 samples)
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train[:1000].reshape(-1, 28 * 28) / 255.0
y_train = y_train[:1000]
class_assignment = np.zeros(y_train.shape[0])
class_assignment[(y_train == 4) | (y_train == 9)] = 1

# Construct the graph signal matrix S based on class assignment
S = class_assignment

# Reshape the images to 1D vectors
x_train_1d = x_train.reshape(-1, 28 * 28)

# Construct the adjacency matrix (1D chain graph) in sparse format
N = x_train_1d.shape[0]  # Number of nodes
adjacency = diags([1], [1], shape=(N, N)) + diags([1], [-1], shape=(N, N))
adjacency = adjacency.tocsr()

# Calculate the inverse square root of the sum of adjacency matrix rows
inv_sqrt_sum_rows = 1 / np.sqrt(adjacency.sum(axis=1)).A.flatten()

# Create the diagonal matrix D_inv_sqrt with the calculated values
D_inv_sqrt = diags([inv_sqrt_sum_rows], [0], shape=(N, N))

# Normalize adjacency matrix
Anorm = D_inv_sqrt @ adjacency @ D_inv_sqrt

# Define your optimization objective and gradient
def cost_function(S, alpha, M1, M2):
    CS_known = diagonal_matrix_C @ S
    CS_algorithm = C @ S
    diff_CS = CS_known - CS_algorithm
    J = 0.5 * (np.linalg.norm((S - (M1 + 2 * alpha * M2) @ S) @ Anorm)**2 +
               alpha * np.linalg.norm(diff_CS)**2)
    return J

def gradient(S, alpha, M1, M2):
    CS_known = diagonal_matrix_C @ S
    CS_algorithm = C @ S
    gradient_J = (M1 + 2 * alpha * M2) @ S - CS_known + CS_algorithm
    return gradient_J

# Define optimization function for scipy.optimize
def optimization_function(S):
    return cost_function(S, alpha, M1, M2), gradient(S, alpha, M1, M2)

# Set initial hyperparameters
alpha = 0.1
epsilon = 1e-6
max_iterations = 1000

# Define C matrix based on class assignment
C = np.diag(class_assignment)

# Define diagonal_matrix_C as a global variable
diagonal_matrix_C = np.diag(S)

# Precompute matrices M1 and M2
M1 = (np.eye(S.shape[0]) - Anorm).T @ (np.eye(S.shape[0]) - Anorm)
M2 = C.T @ C

# Perform optimization with iterative updates
for iteration in range(10):  # Example: 10 iterations, you can adjust as needed
    result = minimize(optimization_function, S, method='L-BFGS-B', jac=True,
                      options={'maxiter': max_iterations, 'gtol': epsilon})
    
    # Extract the optimized signal matrix S and other relevant results
    optimized_S = result.x
    alpha *= 0.9  # Example: Reduce alpha by a factor, you can adjust
    epsilon *= 0.9  # Example: Reduce epsilon by a factor, you can adjust
    
    print(f"Iteration {iteration + 1}:")
    print("Optimized Signal Matrix S:")
    print(optimized_S)
    print(f"Updated alpha: {alpha}")
    print(f"Updated epsilon: {epsilon}")


Iteration 1:
Optimized Signal Matrix S:
[-6.01668867e-04 -8.34960868e-04 -7.08548120e-04 -6.09952495e-04
 -3.13377802e-04 -9.13577258e-05 -3.08951216e-05 -6.17312440e-05
  6.56739702e-05  7.85320979e-04  2.29546141e-03  4.18909182e-03
  5.88431591e-03  6.92340892e-03  7.09489706e-03  6.44435642e-03
  5.27697606e-03  3.78965292e-03  2.19778885e-03  7.56190309e-04
 -1.08596628e-04 -4.56216334e-04 -4.72337864e-04 -4.82584956e-04
 -5.86300421e-04 -8.46498843e-04 -1.37680317e-03 -2.48430806e-03
 -3.48407292e-03 -3.99762983e-03 -3.73172212e-03 -2.79006901e-03
 -1.29978244e-03  4.19141078e-04  2.17077789e-03  3.64972699e-03
  4.66254384e-03  5.34061507e-03  5.59917101e-03  5.38908151e-03
  4.56005652e-03  3.27239342e-03  1.87023994e-03  7.93343328e-04
  3.89304918e-04  3.36578937e-04  4.42706056e-04  4.51028621e-04
  3.33924094e-04  3.06657905e-04  3.28880943e-04  4.04831687e-04
  3.80874777e-04  2.02793417e-04 -9.20891399e-05 -4.00294011e-04
 -5.23065245e-04 -4.42707566e-04 -3.68587764e-04 -

Iteration 2:
Optimized Signal Matrix S:
[ 5.50640522e-03  8.58591142e-03  9.19717794e-03  1.88941706e-02
  3.29999401e-02  5.00280106e-02  5.72261758e-02  4.70362705e-02
  3.42310533e-02  1.61999406e-02  1.46963983e-02  1.38839444e-02
  2.38685763e-02  3.14956740e-02  3.94510276e-02  3.30983664e-02
  2.25685667e-02  4.41120691e-03 -3.62680756e-03 -7.77211442e-03
 -3.53670085e-03 -7.56167203e-04  2.05734977e-03  6.94619519e-03
  1.13035701e-02  2.34511384e-02  3.54452782e-02  5.77268427e-02
  6.64440649e-02  6.36675220e-02  5.03598994e-02  3.70812232e-02
  2.38581438e-02  1.56295281e-02  1.50189355e-02  1.26106755e-02
  1.47754930e-02  2.60833767e-02  4.31453329e-02  5.24868502e-02
  3.80552553e-02  9.15704089e-03 -1.38192680e-02 -1.94680253e-02
 -1.75081215e-02 -8.64281917e-03 -5.77678279e-03  1.00977037e-03
  8.51152988e-03  3.04413009e-02  5.14930661e-02  6.60934108e-02
  4.95282109e-02  1.35343477e-02 -2.78330084e-02 -5.23687030e-02
 -5.14340731e-02 -2.86734674e-02 -1.61461085e-02 -

Iteration 3:
Optimized Signal Matrix S:
[-6.03182226e-06 -6.72757300e-06 -1.72852429e-07  5.79255055e-06
  1.14873590e-05  1.62898806e-05  1.75451532e-05  1.43143109e-05
  8.88982825e-06  6.65698750e-06  8.52027330e-06  1.26080040e-05
  1.04433257e-05  4.03359208e-06 -6.47748946e-06 -1.25263399e-05
 -1.68620582e-05 -1.67188308e-05 -1.58758809e-05 -9.77381069e-06
 -3.58632595e-06  1.55358096e-06  3.83629397e-06  1.53164385e-06
 -3.99945358e-07 -4.24990240e-06 -7.65348162e-06 -1.67951136e-05
 -2.54213746e-05 -3.15323950e-05 -3.22733399e-05 -2.76440227e-05
 -1.97604959e-05 -7.70152414e-06  3.35823727e-07  6.17674518e-06
  4.25622373e-06  3.00916447e-07 -5.18729678e-06 -8.08987215e-06
 -1.17719776e-05 -1.37853037e-05 -1.37114756e-05 -8.14614981e-06
 -2.89850618e-06  1.44882147e-06  2.48302954e-06  7.49229327e-07
 -1.44767111e-06 -2.72055484e-06 -2.13602651e-06 -8.24121815e-07
 -2.32030235e-06 -8.00230056e-06 -1.96107480e-05 -3.00342705e-05
 -3.02026946e-05 -1.75845255e-05 -9.01127450e-06 -

Iteration 4:
Optimized Signal Matrix S:
[-1.35741093e-02 -1.80891544e-02 -4.71380914e-03  1.35416840e-03
  8.74660404e-03  9.59976992e-03  1.19069763e-02  1.19463487e-02
  1.38379969e-02  1.53779140e-02  2.06438439e-02  2.62165021e-02
  2.89669011e-02  2.98596842e-02  2.56004659e-02  2.10844032e-02
  1.56154937e-02  1.23764099e-02  1.00359861e-02  6.99514148e-03
  4.07186643e-03 -1.29761029e-03 -7.18859630e-03 -1.41569990e-02
 -1.18884239e-02 -4.29674848e-03  1.32170528e-02  2.86923579e-02
  4.42843359e-02  5.14997599e-02  5.04677054e-02  3.82853819e-02
  1.81273858e-02  4.71116890e-03  4.72889432e-03  1.81667061e-02
  3.14138399e-02  3.73912257e-02  3.33151094e-02  2.68429915e-02
  2.06840834e-02  1.78234354e-02  1.35389757e-02  5.09093102e-03
 -5.48556453e-03 -1.39043962e-02 -1.62143616e-02 -8.88449085e-03
  5.24646221e-03  1.75005502e-02  2.41927558e-02  1.94751196e-02
  1.10533494e-02 -3.02035366e-04 -5.42399652e-03 -8.06168794e-03
 -6.53402385e-03 -3.56963566e-03 -3.83501276e-03 -

Iteration 5:
Optimized Signal Matrix S:
[-3.08113055e-05 -3.83336577e-05 -1.92809759e-05  4.91357061e-06
  3.75169011e-05  6.95737178e-05  8.18193007e-05  5.83252826e-05
 -1.72701307e-05 -1.54026519e-04 -3.72904872e-04 -6.02957662e-04
 -7.96853214e-04 -9.20338850e-04 -9.58640868e-04 -9.08112757e-04
 -7.72530763e-04 -5.72060025e-04 -3.48559236e-04 -1.49629508e-04
 -3.50377675e-05  1.61094624e-05  3.24398853e-05  3.61384422e-05
  3.41007693e-05  3.47692535e-05  4.64222394e-05  8.17513186e-05
  1.25240383e-04  1.64049394e-04  1.81313283e-04  1.51617070e-04
  6.35846801e-05 -8.84437646e-05 -2.96773605e-04 -5.13124532e-04
 -6.99685254e-04 -8.23233783e-04 -8.59179054e-04 -8.00161532e-04
 -6.61068710e-04 -4.72593764e-04 -2.77260139e-04 -1.07992938e-04
 -1.45835943e-05  2.81784195e-05  4.03789670e-05  3.77895204e-05
  3.71624165e-05  4.17154839e-05  4.67845772e-05  4.47648798e-05
  3.78244685e-05  3.11143220e-05  3.07808443e-05  3.69696188e-05
  3.69628058e-05  2.25977513e-05 -9.01419556e-08 -

Iteration 6:
Optimized Signal Matrix S:
[-2.60485393e-04 -2.45529993e-04 -2.31089025e-04 -1.33893429e-04
 -7.25490110e-05  6.25389892e-05  2.22944144e-04  3.36990553e-04
  7.10767650e-04  1.68479867e-03  3.77964014e-03  6.13740303e-03
  8.29633604e-03  9.59025221e-03  1.01502996e-02  9.60705039e-03
  8.10037673e-03  5.77665521e-03  3.33703235e-03  1.45098802e-03
  4.79523244e-04  7.85578986e-05 -1.80525853e-04 -4.11726339e-04
 -6.72580557e-04 -1.01861489e-03 -1.76559217e-03 -2.99754947e-03
 -4.26737947e-03 -5.02370138e-03 -4.90704383e-03 -3.79579577e-03
 -1.85417995e-03  4.14046303e-04  2.67697591e-03  4.73727316e-03
  6.58424760e-03  8.06344127e-03  8.83070806e-03  8.51666645e-03
  7.34115687e-03  5.55587376e-03  3.71262630e-03  2.11437351e-03
  1.20479663e-03  6.71341877e-04  3.07483780e-04  1.10413352e-04
  1.47360923e-04  6.40792034e-04  1.22980867e-03  1.56643942e-03
  1.32032177e-03  9.50105823e-04  8.86331077e-04  1.11237327e-03
  1.10963590e-03  6.34637046e-04  1.66835487e-04 -

Iteration 7:
Optimized Signal Matrix S:
[-6.73836603e-05 -7.60296361e-05 -3.84311698e-05 -1.89301183e-06
  2.62107062e-05  6.80737432e-05  1.10967361e-04  1.45216398e-04
  1.44352682e-04  1.36069045e-04  1.44452422e-04  1.72006309e-04
  1.66979093e-04  1.30246688e-04  5.16660636e-05 -1.41010355e-05
 -4.79811381e-05 -4.77522042e-05 -3.11990029e-05 -2.19620283e-05
 -1.71695519e-05 -6.24208719e-06 -4.23978393e-06  4.28741884e-06
 -4.16959987e-05 -8.64980676e-05 -1.75280286e-04 -2.55130532e-04
 -3.25167098e-04 -3.56993209e-04 -3.47510779e-04 -3.03968376e-04
 -1.92399723e-04 -6.48572701e-05  4.75012455e-05  1.10649785e-04
  1.11811118e-04  1.04070737e-04  6.76080516e-05  5.48950191e-05
  1.46626515e-05 -1.27634437e-05 -3.09935457e-05 -2.16492899e-05
  8.79986918e-06  2.33603019e-05  2.35053028e-05  5.28480117e-06
 -1.12204936e-05 -4.33386739e-06 -6.55500750e-06 -5.26490789e-06
 -1.93485913e-05 -1.12092560e-05  6.22865687e-06  2.14601026e-05
  1.76403310e-05 -5.81435171e-06 -2.59195231e-05 -

Iteration 8:
Optimized Signal Matrix S:
[-1.00585546e-03 -1.12364865e-03 -6.40551751e-04 -3.02404572e-04
 -4.10030411e-04 -7.26167628e-04 -9.65936012e-04 -7.71509871e-04
  2.10813357e-04  2.27199408e-03  5.70258769e-03  9.52789945e-03
  1.27316192e-02  1.45941897e-02  1.47585172e-02  1.35811034e-02
  1.12197588e-02  8.32593291e-03  5.22321729e-03  2.61645571e-03
  1.06384657e-03  2.03632142e-04 -3.80268571e-04 -9.77091377e-04
 -1.25980786e-03 -1.27816249e-03 -1.10806866e-03 -1.06020762e-03
 -1.00755190e-03 -5.55172119e-04  1.11185453e-04  1.13551155e-03
  2.20467236e-03  3.78387935e-03  6.30802551e-03  8.99044080e-03
  1.11639574e-02  1.23237796e-02  1.25923889e-02  1.21061756e-02
  1.09585192e-02  9.14793802e-03  6.60880605e-03  4.05297415e-03
  2.20428008e-03  9.85611560e-04  9.16013968e-05 -4.77512473e-04
 -7.76420379e-04 -7.79597198e-04 -4.97172192e-04 -1.74078115e-04
  2.24328332e-04  8.10563321e-04  1.78631603e-03  2.81167497e-03
  3.08239654e-03  2.53730241e-03  1.89042078e-03  

Iteration 9:
Optimized Signal Matrix S:
[-5.24681898e-03 -8.04004213e-03 -1.29237714e-02 -1.46351902e-02
 -1.29751035e-02 -5.74322478e-03  2.43174377e-03  9.83770965e-03
  1.39008467e-02  1.79212069e-02  2.50994974e-02  3.39956734e-02
  3.76798854e-02  3.76159861e-02  3.12183504e-02  2.72282828e-02
  2.21080468e-02  1.66353263e-02  7.86017531e-03 -1.57995315e-03
 -6.28724857e-03 -7.37317732e-03 -6.23204047e-03 -7.12699326e-03
 -6.87027343e-03 -2.45417675e-03  8.68324834e-03  2.19591233e-02
  3.31734931e-02  4.06707231e-02  4.34494886e-02  4.27182439e-02
  3.57904568e-02  2.91008352e-02  2.56194344e-02  2.64564916e-02
  2.62098446e-02  3.07605406e-02  3.40240272e-02  3.92199791e-02
  3.32329387e-02  2.35083881e-02  6.92697350e-03 -1.79139304e-03
 -6.20197002e-03 -1.40908168e-03  1.84403443e-03  3.83356948e-03
  4.85527700e-04 -2.18015329e-03 -9.95364443e-04  2.11910555e-03
  6.81092114e-03  6.04931659e-03  4.18799716e-03 -4.02363610e-04
 -3.77313726e-03 -3.60643710e-03 -1.85835073e-03  

Iteration 10:
Optimized Signal Matrix S:
[-3.87295945e-03 -6.18543903e-03 -1.22943543e-02 -1.42775937e-02
 -1.28198296e-02 -4.55315006e-03  4.64500886e-03  1.30136354e-02
  1.75015747e-02  2.15765894e-02  2.89002261e-02  3.80363860e-02
  4.14987852e-02  4.11515756e-02  3.36970355e-02  2.90941595e-02
  2.30698034e-02  1.67947934e-02  7.24854983e-03 -2.85379230e-03
 -7.40520695e-03 -8.06726140e-03 -6.36376903e-03 -7.02741213e-03
 -6.41005655e-03 -1.28507886e-03  1.14868510e-02  2.61689069e-02
  3.81600400e-02  4.53420115e-02  4.72997146e-02  4.57851656e-02
  3.82233490e-02  3.12276796e-02  2.75469651e-02  2.84981661e-02
  2.82959375e-02  3.33897351e-02  3.69438239e-02  4.27389587e-02
  3.59360208e-02  2.51726634e-02  6.82826037e-03 -2.72923848e-03
 -7.41770555e-03 -2.01683425e-03  1.99773073e-03  4.94692177e-03
  2.45454278e-03  6.97976161e-04  2.67871146e-03  5.36312555e-03
  9.07178812e-03  5.88966896e-03  1.63283691e-03 -5.42243685e-03
 -9.88742255e-03 -9.19696500e-03 -6.79164143e-03 